# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921670054968                   -0.69    5.9         
  2   -7.926164532271       -2.35       -1.22    1.0    228ms
  3   -7.926838066190       -3.17       -2.37    1.8    256ms
  4   -7.926861488261       -4.63       -3.06    2.5    314ms
  5   -7.926861653208       -6.78       -3.47    2.2    229ms
  6   -7.926861673473       -7.69       -3.90    2.0    224ms
  7   -7.926861679956       -8.19       -4.23    1.2    206ms
  8   -7.926861681591       -8.79       -4.65    1.6    258ms
  9   -7.926861681857       -9.58       -5.29    2.0    230ms
 10   -7.926861681871      -10.85       -5.75    2.2    239ms
 11   -7.926861681873      -11.72       -6.91    1.2    213ms
 12   -7.926861681873      -13.24       -7.55    3.2    295ms
 13   -7.926861681873   +  -14.57       -7.85    2.4    238ms
 14   -7.926861681873      -14.57       -8.77    1.4    211ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921694174914                   -0.69    5.8         
  2   -7.926162037828       -2.35       -1.22    1.0    217ms
  3   -7.926836650645       -3.17       -2.37    1.8    248ms
  4   -7.926861504938       -4.60       -3.02    2.8    324ms
  5   -7.926861634732       -6.89       -3.36    1.9    228ms
  6   -7.926861667098       -7.49       -3.73    1.8    220ms
  7   -7.926861680606       -7.87       -4.39    1.1    217ms
  8   -7.926861681799       -8.92       -4.91    2.1    280ms
  9   -7.926861681862      -10.20       -5.31    1.6    219ms
 10   -7.926861681872      -11.03       -6.00    1.8    222ms
 11   -7.926861681873      -12.10       -6.74    2.0    264ms
 12   -7.926861681873      -13.47       -7.30    2.2    241ms
 13   -7.926861681873      -14.45       -8.14    2.5    241ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921674954980                   -0.69    6.0         
  2   -7.926164802534       -2.35       -1.22    1.0    288ms
  3   -7.926840950946       -3.17       -2.37    1.9    252ms
  4   -7.926864909078       -4.62       -3.05    2.6    270ms
  5   -7.926865057382       -6.83       -3.43    1.9    226ms
  6   -7.926865082112       -7.61       -3.83    1.8    230ms
  7   -7.926865091296       -8.04       -4.28    1.1    243ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
